In [147]:
import pandas as pd
import numpy as np
import math
import operator

In [4]:
data = pd.read_excel("order_data.xlsx")

In [5]:
#将空由0替换
data = data.fillna(0)

In [6]:
data = data[(data.product_id > 0)]
data["product_id"] = data["product_id"].apply(lambda x: int(x))


In [9]:
#过滤一个用户至少买过两个商品的记录
user_data = data.groupby("user_id").size()
user_data = user_data[user_data > 1]
data = data[data.user_id.isin(user_data.keys())]
user_list = data.values.tolist()


In [10]:
len(data)

76121

In [11]:
all_product_id = list(set(data["product_id"].values.tolist()))

In [12]:
len(all_product_id)

19462

In [78]:
product_to_index = {}
index_to_product = {}
for index,value in enumerate(all_product_id):
    product_to_index[value] = index
    index_to_product[index] = value


In [79]:
index_to_product[0]

100499457

In [23]:
#第一步创建用户-物品的倒排索引
user_item_index = {}
for user_id in user_data.keys():
    product_ids = data[data.user_id == user_id]["product_id"].values.tolist()
    for index,value in enumerate(product_ids):
        product_ids[index] = product_to_index[value]
    user_item_index[user_id] = product_ids
    




In [33]:
#第二步创建共现矩阵

product_length = len(product_to_index)
matrix_c = np.zeros((product_length,product_length))

#循环用户-商品倒排索引 对于同一个用户购买的任意的两个商品 在共现矩阵中都要加1
for user_id in user_item_index:
    product_ids = user_item_index[user_id]
    for i,value in enumerate(product_ids):
        if(i < len(product_ids) - 1):
            list_other = product_ids[(i+1):len(product_ids)]
            for second_product_index in list_other:
                matrix_c[value][second_product_index] += 1
                matrix_c[second_product_index][value] += 1



In [44]:
matrix_c[0]



In [64]:
#第三步根据算法得到商品的相似矩阵 算法：cij/sqrt(|N(i)|*|N(j)|)

product_index_count_dic = {}

product_group = data.groupby("product_id").size()
for product_id in product_group.keys():
    product_index_count_dic[product_to_index[product_id]] = product_group[product_id]
    
matrix_w = np.zeros((product_length,product_length))

#共现矩阵大于0的下标list
index_i_list,index_j_list = np.where(matrix_c > 0)
for index,value in enumerate(index_i_list):
    i = value
    j = index_j_list[index]
    score = matrix_c[i][j]/math.sqrt(product_index_count_dic[i] * product_index_count_dic[j])
    matrix_w[i][j] = score
    matrix_w[j][i] = score


In [73]:
a = np.zeros(product_length)
a[1] = 3
a[2] = 4
a[5] = 6
a = (a - np.min(a))/(np.max(a) - np.min(a))

In [74]:
a

array([ 0.        ,  0.5       ,  0.66666667, ...,  0.        ,
        0.        ,  0.        ])

In [75]:
def normalize(value):
    value = (value - np.min(value))/(np.max(value) - np.min(value))
    return value

In [76]:
#第四步创建用户的喜好商品矩阵：并进行归一化
user_like_item_dic = {}
for user_id in user_data.keys():
    user_like_item = data[data.user_id == user_id]
    user_item_like_matrix = np.zeros(product_length)
    for i in range(len(user_like_item)):
        index = product_to_index[user_like_item.iloc[i].product_id]
        user_item_like_matrix[index] = user_like_item.iloc[i].orders_num
    user_like_item_dic[user_id] = normalize(user_item_like_matrix)
    


In [121]:
user_like_item_dic[27]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [112]:
#获得最相似的k个商品
def getMostSimilar(matrix_w,index,k):
    c_list = matrix_w[index]
    similar_item = pd.DataFrame({"value":c_list})
    similar_item = similar_item.sort_values(by="value",ascending=False).iloc[0:k]
    similar_item_dic = {}
    for i in range(len(similar_item)):
        similar_item_dic[similar_item.iloc[i].name] = similar_item.iloc[i].value
    return similar_item_dic

In [113]:
getMostSimilar(matrix_w,0,10)

{12969: 0.0,
 12976: 0.0,
 19454: 0.24253562503633297,
 19455: 0.33333333333333331,
 19456: 0.3779644730092272,
 19457: 0.5,
 19458: 0.57735026918962584,
 19459: 0.57735026918962584,
 19460: 1.0,
 19461: 1.0}

In [114]:
like_list = np.where(user_like_item_dic[753664] > 0)
print(like_list)

(array([ 862, 6749, 6907, 7157], dtype=int64),)


In [148]:
def reommendItem(user_id,matrix_w,user_like_item_dic,k):
    recommend_dic = {}
    user_like_list = user_like_item_dic[user_id]
    user_like_item_index_list = np.where(user_like_list > 0)
    user_like_item_index_list = user_like_item_index_list[0]
    for product_index in user_like_item_index_list:
        like_score = user_like_list[product_index]
        most_similar_item = getMostSimilar(matrix_w,product_index,k)
        for key in most_similar_item.keys():
            if key in user_like_item_index_list:
                continue
            #最终得分是用户对商品的喜欢程度 * 商品的相似程度
            score = like_score * most_similar_item[key]
            if key in recommend_dic.keys():
                score += recommend_dic[key]
            recommend_dic[key] = score
    #返回得分最高的k个商品
    sorted_x = sorted(recommend_dic.items(), key=operator.itemgetter(1))
    sorted_x.reverse()
    return sorted_x[0:k]

In [151]:

recommend_dic = reommendItem(27,matrix_w,user_like_item_dic,10)
recommend_dic

[(7348, 0.35355339059327373),
 (6851, 0.25),
 (9369, 0.25),
 (15451, 0.20952908873087345),
 (12060, 0.20412414523193154),
 (10971, 0.15811388300841897),
 (15736, 0.15617376188860607),
 (14435, 0.15617376188860607),
 (9440, 0.15617376188860607),
 (12248, 0.15617376188860607)]

In [164]:
#第五步给用户推荐商品

def getAllUserRecommend():
    user_recommend = {}
    for user_id in user_like_item_dic.keys():
        print(user_id)
        recommend_dic = reommendItem(user_id,matrix_w,user_like_item_dic,10)
        value = ""
        for key in recommend_dic:
            index = key[0]
            if value == "":
                value += str(index_to_product[index])
            else:
                value += "," + str(index_to_product[index])
        user_recommend[user_id] = value
    return user_recommend


    


In [165]:
user_recommend = getAllUserRecommend()

753664
360449
229380
458757
655361
327688
688139
327692
688141
622606
688143
131088
65553
294930
688147
229401
524314
27
32796
196637
491552
720930
759131
589861
469681
753705
131114
65579
557101
753710
32816
688177
98355
622644
65589
758337
229431
655416
688185
557114
98363
131133
458817
720907
426052
720966
458764
524362
229451
360524
751791
655438
726371
655444
393301
707055
589914
491612
622685
741047
753761
655458
98405
720998
491623
98409
98410
756354
688236
327698
98414
305520
540691
688244
458871
714694
707703
98428
524414
688255
131201
720150
426115
524422
524423
786568
688266
704535
295053
738691
360592
65682
426132
447854
150
557081
786584
589977
295067
721052
157
655518
98463
229536
786593
524450
785336
735427
300401
753832
655529
589996
229551
688304
721073
524466
327861
688311
393400
229561
146648
189
32958
688319
688322
131267
65732
458949
753863
688328
715468
721100
295120
622801
655572
131285
557270
557271
786648
622809
218
219
721118
753701
753888
715472
721124
753893

231368
690121
722890
395211
722893
591824
722897
591826
562851
100309
133078
650233
559067
755676
690143
788448
788450
133091
677542
133094
722920
690153
710393
296940
362477
722926
460783
722928
251560
721235
559092
362486
722935
624633
322559
750902
100351
395265
737622
133128
460809
296971
722956
428045
779719
690193
231446
329751
296985
133147
133148
679843
296990
133151
722976
657442
624675
140825
67621
231462
624682
428077
722991
690228
788533
100406
526391
464564
329786
591931
297020
125962
624703
109579
788548
788550
133191
493640
329805
788070
297040
428114
690259
622827
486414
493656
788569
587099
690270
67681
460898
526435
329828
100454
591976
591981
788590
693949
329840
755825
683027
460917
755831
231544
591993
428154
329851
624764
166013
493695
755843
781334
329862
67719
624777
493707
644802
690323
362645
329880
329881
723098
690331
754010
264351
723107
788644
475505
657576
133295
526513
657587
67764
657591
788664
690361
657596
744225
690366
559295
624832
592065
592068
657

200709
692230
692235
725004
768505
717247
102419
36885
692249
528410
102427
692252
561182
364575
69666
69668
659493
692263
495660
200749
626734
503133
561202
475827
626741
102454
710665
495673
397371
736651
692287
650592
364611
692295
69704
694284
528458
168013
561232
102481
495698
725075
692309
725079
626776
528473
725086
725088
692321
692322
574139
692324
233573
692326
69736
102507
725100
299117
594030
135281
659570
757875
462965
626806
462967
736653
692345
397435
594044
740005
541376
626818
69763
717114
233605
355692
689857
102541
331918
495759
725136
135313
679143
528532
339310
430231
594074
495771
692381
626847
781680
331938
757925
725159
743452
721607
311114
765298
331950
561329
299186
495796
135350
606921
322932
777027
495803
561340
692413
191861
579616
782313
594115
725189
364744
725193
561355
692428
426701
659664
725203
463061
430294
692439
692441
784666
757983
626913
430307
430308
694310
561383
102632
692458
495851
299247
561392
725234
692468
692469
725239
332025
659706
69247

694313
563244
563245
137263
530480
497713
754177
694326
700084
785820
366943
333884
38975
694336
694337
38978
333891
563268
732355
530510
115725
530515
694356
432213
432214
634553
735629
530523
366686
727136
694369
727138
732859
661604
692954
596072
694378
454333
497776
694385
301170
759925
301174
694391
6265
759635
465020
252266
694398
771093
727169
661635
170116
661639
694408
399497
705559
170124
301197
399503
727184
661649
71826
742660
694421
628888
661657
301210
767274
497820
137373
465054
628895
628897
497827
366758
563368
727209
727212
39087
497840
241352
759986
661684
656414
694454
71866
71867
104636
707263
334016
694465
530626
596163
104644
694469
268487
762911
727242
39119
661712
547192
684350
536270
563414
246820
759682
563422
754725
399584
747636
22907
39140
727272
530666
628971
753092
628974
301296
727285
295977
97807
465145
681611
61141
661760
399744
71938
170243
629000
694540
104717
596239
596241
694547
596244
629014
727319
334104
763174
71962
694555
683738
366878
760095


696429
760509
598128
176147
532596
640361
696441
694975
467068
106624
630913
139394
565379
204933
172168
729225
729226
729227
336012
492377
558445
172176
630929
598164
722286
565398
684057
467098
312687
73887
654956
41124
73893
686851
434347
303276
374536
709513
630962
623987
696500
598198
565432
565433
598202
716831
73918
765984
696514
565443
762052
700449
727756
434378
762059
565453
729294
563917
663760
41169
598226
729299
565463
696538
139483
591226
237791
762083
565476
336101
696550
780347
565482
499947
303340
498386
106735
700456
336114
760531
139510
368888
631033
467194
762108
258090
252629
73984
722304
631042
663811
778257
434438
787841
303368
139530
729355
696589
696590
749613
563928
663826
558467
663829
499990
467223
729368
705924
782383
696604
499997
754253
598305
270627
729380
324348
651313
663848
663852
467245
629469
631095
746821
687695
729404
624010
663873
771566
700470
532806
711393
696648
237897
336202
303435
696654
307256
689550
434519
336216
434521
510933
525711
76220

...

613180
449342
282421
678721
252738
645955
711494
514891
252748
252749
121678
449362
770025
580436
580438
708409
580440
728527
138261
775481
678750
777055
449377
645987
23396
482151
645993
547690
678764
725316
711536
252785
711538
121717
318326
711544
744314
632544
613247
711553
154499
697494
708417
678793
744330
547723
547724
449424
711570
318355
744340
613269
583444
549619
121756
613277
698013
482209
220067
449445
482215
580520
318378
678373
318380
736670
547759
708424
727078
781182
285621
89014
220090
547771
383932
613310
547775
730272
547781
678854
768503
580556
744397
737319
528205
580562
681123
547799
154585
285660
678877
383968
298832
711653
777190
515047
711656
711658
580587
154605
706986
768509
154611
646134
364372
646142
580607
121856
711681
646151
711689
560983
318476
482319
777232
675672
89106
80027
154648
686596
777244
675674
56351
449570
580643
613412
613415
384043
713906
678959
646192
763059
384053
236470
683200
515129
708447
744508
646207
56384
154689
56386
784907
777285

451390
90945
648003
549700
418629
222022
189921
582473
451402
713548
697826
86157
648017
680787
713556
708750
353110
418649
353114
713563
615261
287583
353120
516962
746342
615275
666409
727540
648680
713586
713587
123764
686911
713596
713597
648065
156547
615300
720994
516998
555841
713609
680842
626839
385933
222095
490766
25495
713624
287641
287642
320411
549789
473925
704628
320420
746993
713640
746409
710084
287661
713646
451503
648112
713650
582580
680887
91064
549818
752458
648126
549823
353217
703307
713670
774305
713672
91081
779210
692627
287693
680911
484304
747000
615379
736778
287705
484314
730618
746462
708773
91105
582627
779238
746471
713706
418795
746476
648174
222192
353267
713716
189429
353270
549881
648186
740339
713727
680960
757931
320517
713734
648199
320522
697858
386062
680975
768856
353299
713750
736089
648216
599197
713755
713756
648222
58885
91168
58401
484394
582702
517168
705724
746548
25653
615484
320573
90327
713791
762211
738861
776648
741558
715511
156

125782
485177
326799
486236
125789
604665
158559
682849
603219
322403
617316
125797
736337
486249
256875
179346
747325
420721
158579
158581
322422
545257
617336
125818
772345
781180
519038
60287
712333
748418
715652
687254
125831
781192
189682
682892
224142
584591
191377
125842
324961
305043
158616
715674
715675
519068
524577
781220
743977
551846
744111
715691
584621
551856
715698
715701
698185
676340
748474
650171
762527
486334
283125
584641
715714
355269
355271
715720
617418
692727
682956
785570
715728
336987
748501
584662
289751
748505
158682
682971
725498
289758
748512
730960
650214
682983
584680
584683
747346
715758
698195
650229
682998
617465
93179
715772
715774
617471
785443
388099
388100
698198
683014
781320
224266
763735
463362
617486
425133
125968
486418
659971
322581
388118
748568
146607
617501
748574
715807
224288
125985
486435
93223
748585
584747
453679
748593
720051
551988
709196
715836
60477
355390
683071
715841
683074
748611
486469
748614
388167
773443
744985
158798
552

717638
357191
62281
488269
488272
160593
619346
652115
586581
324438
430905
684888
736740
62299
720357
652128
619361
488290
742204
717674
736743
455532
652141
521070
324463
95088
606861
521074
750452
652149
95096
141460
770983
488315
684924
769140
127874
652163
455559
717708
488333
652175
291730
578371
768408
632985
258969
783258
390043
698522
160670
652191
455584
455586
324515
541095
717734
753137
559087
422828
223389
690005
390065
688041
496457
357305
357312
160705
586691
687606
684998
127943
160712
291787
259020
455629
488399
717777
160723
521172
127958
324568
521179
127967
717792
95205
127976
294396
750574
717807
127984
338088
652274
685044
390133
521208
685049
717819
455676
717821
390142
29695
324608
586754
332630
324614
488455
128009
487633
783371
455692
128013
452781
685074
324627
742816
128029
455710
586783
717857
455714
717859
554020
62501
717863
697131
586794
455723
750636
226351
669853
458248
554034
693508
652340
652341
95286
160823
726592
591292
619581
226366
750657
673127


293679
677000
696100
737076
693386
326462
64319
293697
719682
780769
654154
719691
621391
487695
162641
392018
719699
758926
686935
392767
761935
326493
155992
523105
625178
162659
490340
457573
555878
654183
621416
782301
689555
775132
359280
621425
686967
31610
621440
523139
457604
785285
718824
392072
469484
606018
588686
781829
686994
91617
97172
162709
752535
752537
779933
490396
293790
551408
555940
767335
687017
719786
769863
293804
424878
555951
588725
245577
555960
752570
122170
549893
621501
719806
567797
752578
654276
555975
555976
490441
129997
555982
31697
555986
588756
490454
248742
490456
698874
31713
162789
523240
729931
588781
457710
780797
130035
556020
687093
752631
687101
687103
770113
457734
461514
556042
785423
130064
588817
97298
719891
742574
293910
359447
457752
694783
719901
621599
719907
130084
31781
785447
588840
588843
523310
638813
307666
359473
523318
453129
752697
294751
761379
359488
752705
687171
726198
425032
780812
490571
31821
698893
677048
343907
6

In [166]:
user_recommend

{753664: '100239043,100290978,100476538,100279433,100238902,100474580,100239202,100475860,100073762,100500821',
 360449: '100453984,100167564,100146130,100479007,100144377,100142780,100142744,100424669,100424921,100383704',
 229380: '100508560,100251783,100485082,100210935,100498247,100489722,100381230,100484665,100457053,100316008',
 458757: '100079888,100499234,100321484,100075708,100073892,100175761,100175580,100210725,100453983,100189767',
 655361: '100511045,100465246,100502961,100143015,100357123,100470314,100404535,100497970,100391556,100467555',
 327688: '100405917,100067523,100174010,100176881,100501832,100154820,100406879,100144055,100152579,100341328',
 688139: '100070855,100146041,100142266,100503260,100077641,100474244,100461314,100474736,100478922,100217050',
 327692: '100199788,100387402,100473651,100466942,100477667,100207308,100421134,100219512,100175104,100323010',
 688141: '100496955,100356013,100522260,100498051,100509624,100217390,100480725,100147932,100277634,1001

In [173]:
user_like_list = user_like_item_dic[327692]
user_like_item_index_list = np.where(user_like_list > 0)
value = ""
for key in user_like_item_index_list[0]:
    value += "," + str(index_to_product[key])
print(value)

,100172313,100381306,100167483


In [59]:
i,j = np.where(matrix_c > 0)

In [60]:
i

array([    0,     0,     0, ..., 19461, 19461, 19461], dtype=int64)

In [61]:
j

array([ 1915,  2099,  2622, ..., 14405, 15998, 17857], dtype=int64)